In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
from matplotlib import pyplot as plt
import tqdm

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
oil_data = pd.read_csv('data/oil.csv')

In [3]:
oil_table = pd.DataFrame()
oil_table['date'] = sorted(list(train_data.date.unique()) + list(test_data.date.unique()))
oil_data = oil_table.merge(oil_data, how='left', left_on='date', right_on='date')
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN
...,...,...
1695,2017-08-27,NaN
1696,2017-08-28,46.40
1697,2017-08-29,46.46
1698,2017-08-30,45.96


In [4]:
for i in range(len(oil_data)):
    if not np.isnan(oil_data.dcoilwtico[i]):
        continue
    if i == 0:
        oil_data.iloc[i, 1] = oil_data.dcoilwtico[i+1]
    elif i == len(oil_data) - 1:
        oil_data.iloc[i, 1] = oil_data.dcoilwtico[i-1]
    else:
        oil_data.iloc[i, 1] = (oil_data.dcoilwtico[i+1] + oil_data.dcoilwtico[i-1]) / 2

In [5]:
oil_data = oil_data.fillna(method='ffill')

In [6]:
# oil_data = oil_data.set_index('date')
# oil_data.dcoilwtico.plot(figsize=(20, 10))
# plt.show()

In [7]:
holiday_data = pd.read_csv('data/holidays_events.csv')
holiday_data.type = holiday_data.type.replace({"Transfer":"Holiday"})
holiday_data['holiday_type'] = holiday_data.type + "_" + holiday_data.locale + "_" + holiday_data.transferred.astype(str)

In [8]:
store_data = pd.read_csv('data/stores.csv')

In [9]:
trans_data = pd.read_csv('data/transactions.csv')

In [10]:
trans_data

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [11]:
train_data['month'] = train_data.date.str.slice(start=5, stop=7)
train_data['day'] = pd.to_datetime(train_data.date).dt.dayofweek

In [12]:
train_data

,id,date,store_nbr,family,sales,onpromotion,month,day
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,01,1
1,1,2013-01-01,1,BABY CARE,0.000,0,01,1
2,2,2013-01-01,1,BEAUTY,0.000,0,01,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,01,1
4,4,2013-01-01,1,BOOKS,0.000,0,01,1
...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,08,1
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,08,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,08,1
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,08,1


In [13]:
train_data = train_data.merge(store_data, left_on='store_nbr', right_on='store_nbr', how='left').merge(oil_data, left_on='date', right_on='date', how='left')

In [ ]:
holiday_types = []
for i in tqdm.tqdm(range(len(train_data))):
    date = train_data.iloc[i, 1]
    city = train_data.iloc[i, 8]
    state = train_data.iloc[i, 9]
    
    if date not in holiday_data.date.unique():
        holiday_type = 'Not_Holiday'
    else:
        this_holiday = holiday_data[holiday_data.date == date]
        if "National" in this_holiday.locale.unique():
            holiday_type = this_holiday[this_holiday.locale == 'National'].holiday_type.values[0]
        if "Regional" in this_holiday.locale.unique() and state in this_holiday.locale_name.unique():
            holiday_type = this_holiday[(this_holiday.locale=='Regional')&(this_holiday.locale_name==state)].holiday_type.values[0]
        if "Local" in this_holiday.locale.unique() and city in this_holiday.locale_name.unique():
            holiday_type = this_holiday[(this_holiday.locale=='Local')&(this_holiday.locale_name==city)].holiday_type.values[0]
    holiday_types.append(holiday_type)
train_data['holiday_type'] = holiday_types

100%|█████████████████████████████████████████████████████████████████████▋| 2987590/3000888 [10:57<00:02, 6360.86it/s]

In [ ]:
train_data = train_data.merge(trans_data, how='left', left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'])
train_data.transactions = train_data.transactions.fillna(0)

In [ ]:
train_data

In [ ]:
train_data.to_csv('data/train_data.csv', index=False)

In [ ]:
test_data = pd.read_csv('data/test.csv')

In [ ]:
test_data['month'] = test_data.date.str.slice(start=5, stop=7)
test_data['day'] = pd.to_datetime(test_data.date).dt.dayofweek

In [ ]:
test_data

In [ ]:
test_data = test_data.merge(store_data, left_on='store_nbr', right_on='store_nbr', how='left').merge(oil_data, left_on='date', right_on='date', how='left')

In [ ]:
test_data.isna()

In [ ]:
holiday_types = []
for i in tqdm.tqdm(range(len(test_data))):
    date = test_data.iloc[i, 1]
    city = test_data.iloc[i, 8]
    state = test_data.iloc[i, 9]
    
    if date not in holiday_data.date.unique():
        holiday_type = 'Not_Holiday'
    else:
        this_holiday = holiday_data[holiday_data.date == date]
        if "National" in this_holiday.locale.unique():
            holiday_type = this_holiday[this_holiday.locale == 'National'].holiday_type.values[0]
        if "Regional" in this_holiday.locale.unique() and state in this_holiday.locale_name.unique():
            holiday_type = this_holiday[(this_holiday.locale=='Regional')&(this_holiday.locale_name==state)].holiday_type.values[0]
        if "Local" in this_holiday.locale.unique() and city in this_holiday.locale_name.unique():
            holiday_type = this_holiday[(this_holiday.locale=='Local')&(this_holiday.locale_name==city)].holiday_type.values[0]
    holiday_types.append(holiday_type)
test_data['holiday_type'] = holiday_types

In [ ]:
test_data = test_data.merge(trans_data, how='left', left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'])
test_data.transactions = test_data.transactions.fillna(0)

In [ ]:
test_data

In [ ]:
test_data.to_csv('data/test_data.csv', index=False)